## Problem Statement 1:

We have three workers with identical skills levels (1 unit of work per unit of time).  We have two types of tasks, digging and welding, each requiring 10 units of work.  How long will it take to finish all tasks?

Answer: the first 18 units of work will be completed in 6 time, with all workers working simultaneuously.  The final two units of work will be completed by only two workers.  Total time: 7.

In [24]:
# Worker skills
# Digging | Welding    
    S = [1 1;
         1 1;
         1 1]

# Task work
# Digging | Welding
   T = [10 10]


using JuMP, Gurobi

m = Model(solver=GurobiSolver())

@variable(m, z[1:size(S,1), 1:sum(T)], Bin) #Binary whether a worker works on a task
@variable(m, s[1:size(S,1),1:size(S,2)] == 1) #The skill level of a worker
##@variable(m, times[1:size(S,1),1:size(S,2)]) # The start time of a task

@variable(m, task1[1:T[1]])
@variable(m, task2[1:T[2]])
        
@variable(m, completionTime) # the total time it takes to finsih a project


# If we are at the end time, then all tasks must be completed
#@constraint(m, )

# If a worker is on a task, then they can not be worker on another task
@expression(m,  A = zeros(find(z[1,:], sum(z;1,:]))
                for i=1:size(A,1)
                    for j=1:size(A,2)
                        if z[1,i] == true
                            if 
            )
@constraint(m, z[1,i]* <= 1)

m
                        
                        
#work in progress....

LoadError: [91msyntax: unexpected "]" in argument list[39m

Simplying attempt to one task and creating constraints a worker limited to one task at a time, without using discrete time.

Problem here with comparing quadratic constraints

In [23]:
#number of Workers
n=3
#Number of tasks
T= 4

using JuMP, Gurobi
m = Model(solver=GurobiSolver())
@variable(m, z[1:n,1:T], Bin) #whether worker w  works task t
@variable(m, t[1:T]) #the start time of each task

@variable(m, d1[1:n,1:T], Bin) #Boolean logic variable
@variable(m, d2[1:n,1:T], Bin) #Boolean logic variable
@variable(m, q) # Minimax step for objective
#Each task is worked on once
@constraint(m, taskCompletion[i=1:T], sum(z[:,i]) == 1)

#Each worker works one task at a time
#Each task ends before t[j] or starts after t[j] ends

# is true if each tasks starts after this one ends (assume tasks length is 1, upper time bound of 100)
@constraint(m, a[k=1:T,i=1:n, j=1:T], d1[i,j]*100 >= (z[i,j]*(t[j]+1) - z[i,k]*t[k]))
# is true if each tasks ends before this one starts (same assumptions)
@constraint(m, b[k=1:T,i=1:n,j=1:T],  d2[i,j]*100 >= (z[i,j]*t[j]- z[i,k]*(t[k]+1)))
#use q to minimuze the maximum end time
@constraint(m, [i=1:n,j=1:T], q >= z[i,j]*(t[j] + 1))

@constraint(m, a+b .>= 1) #one of the above statements must be true


@objective(m, Min, q)

status = solve(m)


LoadError: [91mMethodError: no method matching *(::Float64, ::JuMP.ConstraintRef{JuMP.Model,JuMP.GenericQuadConstraint{JuMP.GenericQuadExpr{Float64,JuMP.Variable}}})[0m
Closest candidates are:
  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  *(::Float64, [91m::Float64[39m) at float.jl:379
  *(::Real, [91m::Complex{Bool}[39m) at complex.jl:254
  ...[39m

Reformulating problem to avoid comparing quadartic constraints by using an epigraph trick between start times and end times.

Cannot create a model with a PSD matrix

In [3]:
#number of Workers
n=2
#Number of tasks
T= 3

using JuMP, Gurobi
m = Model(solver=GurobiSolver())
@variable(m, z[1:n,1:T], Bin) #whether worker w  works task t
@variable(m, t[1:T], Int) #the start time of each task


@variable(m, taskStartTime[1:T])
@variable(m, taskEndTime[1:T])
#@variable(m, d1[1:T], Bin)  #generic binary variable for logic

for i=1:n
    for j=1:T
        @constraint(m,[k=1:T], z[i,j]*(t[j]+1) <= taskStartTime[k])
        @constraint(m,[k=1:T], z[i,j]*(t[j]) >= taskEndTime[k])
    end
end


@constraint(m, [k=1:T], taskStartTime[k] == taskEndTime[k] + 1)

@objective(m, Min, sum(taskEndTime))  # generic objective to see if solver reports error

solve(m)


Academic license - for non-commercial use only
Optimize a model with 3 rows, 15 columns and 6 nonzeros
Model has 36 quadratic constraints
Variable types: 6 continuous, 9 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 0

Best objective -, best bound -, gap -
Academic license - for non-commercial use only
Optimize a model with 3 rows, 15 columns and 6 nonzeros
Model has 36 quadratic constraints
Variable types: 6 continuous, 9 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range       

LoadError: [91mGurobi.GurobiError(10020, "Q matrix is not positive semi-definite (PSD)")[39m

In [5]:
m

Minimization problem with:
 * 3 linear constraints
 * 36 quadratic constraints
 * 15 variables: 6 binary, 3 integer
Solver is Gurobi